In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import tensorflow as tf
import pathlib
from tqdm import trange

from src.DataPreprocessor.data_preprocessor import DataPreprocessor, Mode
from src.DataPreprocessor.DataIOBackend.gdal_backend import GdalBackend
from src.DataPreprocessor.data_visualiser import DataVisualiser
from src.LearningKeras.net_architecture import cnn_150x150x5,cnn_150x150x5_3class
from src.LearningKeras.train import KerasTrainer

np.random.seed(1)
tf.set_random_seed(2)

check data_visualisation for details

In [2]:
dataio_backend_1 = GdalBackend()
dataio_backend_2 = GdalBackend()

data_preprocessor_1 = DataPreprocessor(data_dir="data/Region 1 - Lopukangri/",
                              backend=dataio_backend_1,
                              filename_prefix="tibet",
                              mode=Mode.TRAIN,
                              seed=1)

data_preprocessor_2 = DataPreprocessor(data_dir="data/Region 2 - Muga Puruo/",
                              backend=dataio_backend_2,
                              filename_prefix="mpgr",
                              mode=Mode.TRAIN,
                              seed=1)

In [4]:
batch_size = 10

train_generator_1 = data_preprocessor_1.train_generator(batch_size=batch_size,
                                         class_probabilities=np.array([1./3, 1./3, 1./3]),
                                         patch_size=(150, 150),
                                         channels=np.array([0, 1, 2, 3, 4]))

train_generator_2 = data_preprocessor_2.train_generator(batch_size=batch_size,
                                         class_probabilities=np.array([1./3, 1./3, 1./3]),
                                         patch_size=(150, 150),
                                         channels=np.array([0, 1, 2, 3, 4]))


In [5]:
#import threading

#lock = threading.Lock()
def joint_generator():
    while True:
            img_batch_1, lbl_batch_1 = next(train_generator_1)
            img_batch_2, lbl_batch_2 = next(train_generator_2)
            img_joint = np.concatenate((img_batch_1, img_batch_1), axis=0)
            lbl_joint = np.concatenate((lbl_batch_1, lbl_batch_2), axis=0)
            #with lock:
            yield img_batch_1, lbl_batch_1

## NN

### Train

In [12]:
model_generator = lambda: cnn_150x150x5_3class()
ensemble_size = 2
batch_size = 5

trainer = KerasTrainer(model_generator=model_generator,
                       ensemble_size=ensemble_size,
                       data_preprocessor=None,
                       batch_size=batch_size)

In [ ]:

#generator = joint_generator()

history_arr = trainer.train(steps_per_epoch=50, epochs=5, train_generator=joint_generator())

Epoch 1/5
 1/50 [..............................] - ETA: 20:56 - loss: 1.1045 - acc: 0.5000

In [11]:
for i in range(ensemble_size):
    pathlib.Path('models_joint').mkdir(parents=True, exist_ok=True)
    trainer.models[i].save_weights('models_joint/model_{}.h5'.format(i))